In [1]:
#librerias
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
import pandas as pd
from datetime import datetime
import numpy as np
import re

In [2]:
##Nuevas
def buscar(elemento,busqueda):
    WebDriverWait(driver, 10)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento)))\
        .send_keys(busqueda)
    
def Click(elemento):
    WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento))).click()

def clear_buscar(elemento):
    WebDriverWait(driver, 20)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,elemento)))\
        .clear()
    
#Se solicita el Xfullpath para hacer click
def Click_Xpath(path):
    WebDriverWait(driver, 15)\
    .until(EC.element_to_be_clickable((By.XPATH,path)))\
    .click()
    
    
##Funcion NoClick hace referencia a los apartados que no son clikeables y se accede a el por medio del FullXpath
def NoClick(path):
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,path)))
    url=driver.find_element("xpath",path).get_attribute("href")
    driver.get(url)   
    
def ListarNombres(path,clase):
    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH,path)))
    html_list=driver.find_element(By.CSS_SELECTOR,clase)
    lista = html_list.find_elements(By.TAG_NAME,'div')
    titulo=[]
    for i in lista:
        text = i.get_attribute('innerHTML')
        soup = BeautifulSoup(text,'html.parser')
        titulo.append(soup.findAll("h2")[0].string)
        #print(titulo)
    return titulo

def ListarPrecios(path,clase):
    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH,path)))
    html_list=driver.find_element(By.CSS_SELECTOR,clase)
    lista = html_list.find_elements(By.TAG_NAME,'li')
    
    for i in lista:
        text = i.get_attribute('innerHTML')
        soup = BeautifulSoup(text,'html.parser')
        precio = soup.select("span.price-tag-fraction")[0].text
        print(precio)
def LeerJson(path):
    
    WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.XPATH,path)))
    Json_Jumbo=driver.find_element("xpath",path).get_attribute("text")
    Json_Jumbo = json.loads(Json_Jumbo)
    DatosProducto = Json_Jumbo['itemListElement'] # Se toma el Json anidado que contiene los datos del producto
    return DatosProducto

In [3]:
#Funciones para limpieza y procesamiento de datos 
def cantidad(producto):
    
    l=len(producto)
    producto=producto[::-1] #Invertimos la cadena para encontrar x
    m =producto.find("x")
    producto = producto[:m] #Esta invertimo el valor
    producto=producto[::-1] #Lo volvemos a su estado normal
    return producto
    




In [4]:
#Ruta de chorme compatible 
driver_path = "C:\\Users\\danie\\OneDrive\\Documentos\\Notebooks\\chromedriver.exe"
## Opciones con las que inicia google chrome 
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--headless")
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

s = Service(driver_path)#Se agrega el path como objeto para evitar errores 


# Main

## Función buscar:
- Recibe como parametro el input class del cuadro de busqueda de la pagina y el producto a buscar
- Se antepone input antes de colocar el input class
- Si el input class tiene espacios estos se remplazan por puntos

## Funcion Click
- Recibe como parametro el elemento clikeable anteponiendo el formato(div, footer, etc)


## Función ListarProductos
- Esta funcion lista todos los productos de la busquema realizada 


In [8]:
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get("https://www.tiendasjumbo.co/") 

ListaProducto=  ['arroz y granos',     'aseo personal',     'despensa']
CantMas = dict([('arroz y granos', 2),('aseo personal', 2),('despensa', 2)])


Json_Total=[]
time.sleep(3)
for i in range(len(ListaProducto)):
    time.sleep(3)
#Buscar producto
    producto = ListaProducto[i]
    input_class = 'input.vtex-styleguide-9-x-input.ma0.border-box.vtex-styleguide-9-x-hideDecorators.vtex-styleguide-9-x-noAppearance.br2.br-0.br--left.w-100.bn.outline-0.bg-base.c-on-base.b--muted-4.hover-b--muted-3.t-body.pl5'
    buscar(input_class,producto)
    time.sleep(2)
#Click en buscar
    Element_clickeable = 'footer.vtex-search-2-x-tileListFooter'
    Click(Element_clickeable)
    time.sleep(2)
    
#Scroll
    Element_clickeable2 = 'a.vtex-button.bw1.ba.fw5.v-mid.relative.pa0.lh-solid.br2.min-h-small.t-action--small.bg-action-primary.b--action-primary.c-on-action-primary.hover-bg-action-primary.hover-b--action-primary.hover-c-on-action-primary.pointer.inline-flex.items-center.no-underline'
    for j in range(CantMas[ListaProducto[i]]):
        Click(Element_clickeable2)
        time.sleep(2)
#Guardar en Json     
    time.sleep(1)
    
    driver.set_page_load_timeout(30)
    path= '/html/body/div[2]/div/div[1]/div/div[4]/div/div/section/div[2]/div/div[4]/div/div[2]/div/div/div[2]/div/div[5]/div/div/div/script'
    Json_Total += LeerJson(path) #Esto deja un JSON con el que se trabajara los datos
#limpiar
    if(i>=1):
        driver.get("https://www.tiendasjumbo.co/") 
driver.close()
print('Finalizo correctamente el proceso')

Finalizo correctamente el proceso


# Procesamiento de datos

In [9]:
#Se obtienen los datos del segundo nivel de profundidad del json y se pasan a un dataframe
Producto_1 = pd.json_normalize(Json_Total) 
#Se accede al 3 nivel de produndidad del json 
JsonAnidado=[] #Se crea una lista vacia para llenarla con el for

for i in Producto_1['item.offers.offers']:
    JsonAnidado +=i 
    
#Se normaliza el nivel 3 de profundidad del json    
Producto_2= pd.json_normalize(JsonAnidado) # Hacemos join de los 2 dataframe creados
Productos_T = pd.concat([Producto_1, Producto_2], axis = 1) 

In [10]:
#Seleccionamos las columnas que nos interesan
Productos_T = Productos_T[['sku','item.name','item.brand.name','item.description','priceCurrency','price','priceValidUntil','item.@id','item.image','seller.name']]
#Renombramos las columnas 
Productos_T.columns = ['Codigo_Pro','Nombre', 'Marca', 'Descripcion', 'Moneda', 'Precio','Precio_Valido_Hasta','Url_producto', 'Url_Imagen','Almacen']

In [11]:
#Obtenemos la fecha actual
now = datetime.now()
format = now.strftime('%Y-%m-%d')
NombreAlmacen='Jumbo-'+format

In [121]:
#Funcion para convertir palabras con acentos en palabras sin acentos
def categoria(producto):
    
    m =producto.find(" ")
    remplazo = producto[:m].maketrans("áéíóú", "aeiou","*123456789") #Remplazo palabras que contengan acentos y borrar caracteres *123456789
    producto = producto[:m].translate(remplazo).upper() #Uso translate para poder observar correctamente
    return producto

In [199]:
#Funcion para obtener la cantidad de cada producto
def cantidadx(producto):
    
    producto = re.findall(r'( x.+\d{0,6}.\w{1,3}|\d{1,5}..\w{0,5})',producto) #Expresion regular para obtener las cantidades de los productos
    producto = " ".join(producto).replace('x', '') #Eliminamos la x en la cantidad de productos y pasamos los datos a str

    return producto.lower() #Retornar en minusculas y sin espacios al inicio y al final 

#Función para separar el numero de la unidad de medida en el nombre
def separador(producto):
    patron =r'(\d{1,5})([a-z]{0,7})' #Expresion regular para dar espacio entre numeros y letras

    producto = re.sub(patron,r'\1 \2',producto) #Separamos los numeros que estan junto a palabras de unidad de medida

    return producto


# Limpieza de datos

In [197]:

Productos_T['Nombre']=Productos_T['Nombre'].apply(lambda x: separador(x))
Productos_T['Cantidad']=Productos_T['Nombre'].apply(lambda x: cantidadx(x))
Productos_T['Fecha']= now.strftime('%Y-%m-%d')
Productos_T['Categoria']=Productos_T['Nombre'].apply(lambda x: categoria(x))

## Exporta archivo json


In [14]:
#Exportar archivo JSON
Productos_T.to_json(f'{NombreAlmacen}.json', orient = 'split', compression = 'infer', index = 'true') 

In [128]:
Productos_T.to_csv(f'{NombreAlmacen}.csv', index=False,sep='|',encoding='utf-8')

In [191]:
producto ='Aceite Riquisimo garrafa 5000 ml'

cantidadx(producto)



['5000 ml']

In [198]:
Productos_T

,Codigo_Pro,Nombre,Marca,Descripcion,Moneda,Precio,Precio_Valido_Hasta,Url_producto,Url_Imagen,Almacen,Cantidad,Fecha,Categoria
0,22018,Arroz vitarroz plus Supremo x 10 kg,SUPREMO,"Arroz Supremo Premium es símbolo de la excelencia del arroz Colombiano, llevalo a tu casa con tan solo un click en tiendasjumbo.co",COP,36590,2023-08-28T22:06:36Z,https://www.tiendasjumbo.co/arroz-vitarroz-plus-supremo-x-10-kg/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/192008/7701024000276-1.jpg?v=1751413508,Cencosud - Colombia,[ x 10 kg],2022-10-05,ARROZ
1,111865,Arroz Cuisine&amp;Co blanco x10000 g,CUISINE &amp; CO NBE MP,Arroz cuisine&amp;co blanco x10000g,COP,37490,2023-08-28T22:30:36Z,https://www.tiendasjumbo.co/arroz-cuisine-co-blanco-x10000g/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/213565/7703616303888.jpg?v=1751415952,Cencosud - Colombia,[ x10000 g],2022-10-05,ARROZ
2,13397,Arroz Diana x 10 Kg,DIANA,Proporciónale todo el amor a tu familia con el delicioso Arroz Diana,COP,36890,2023-09-27T00:00:00Z,https://www.tiendasjumbo.co/arroz-diana-x-10-kg/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/186325/7702511000205.jpg?v=1751415990,Cencosud - Colombia,[ x 10 Kg],2022-10-05,ARROZ
3,13396,Arroz Diana x 5 Kg,DIANA,Proporciónale todo el amor a tu familia con el delicioso Arroz Diana,COP,19490,2023-08-28T22:22:13Z,https://www.tiendasjumbo.co/arroz-diana-x-5-kg/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/186323/7702511000045.jpg?v=1751413572,Cencosud - Colombia,[ x 5 Kg],2022-10-05,ARROZ
4,22006,Arroz Supremo x 5 kg,SUPREMO,"Arroz Supremo contiene una mezcla de vitaminas y minerales que contribuyen a la nutrición de tu familia.Contribuye al crecimiento y desarrollo del cuerpo y, ayuda a fortalecer el sistema inmunológico.Contribuye al buen funcionamiento del sistema inmunológico.",COP,19590,2023-06-12T22:04:08Z,https://www.tiendasjumbo.co/arroz-supremo-x-5-kg/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/191981/7701024000047-1.jpg?v=1751416216,Cencosud - Colombia,[ x 5 kg],2022-10-05,ARROZ
5,13395,Arroz Diana x 3 Kg,DIANA,Proporciónale todo el amor a tu familia con el delicioso Arroz Diana,COP,11690,2023-08-28T22:06:26Z,https://www.tiendasjumbo.co/arroz-diana-x-3-kg/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/186321/7702511000038.jpg?v=1751413632,Cencosud - Colombia,[ x 3 Kg],2022-10-05,ARROZ
6,13376,Arroz Diana x 25 und x 500 g c-u,DIANA,Proporciónale todo el amor a tu familia con el delicioso Arroz Diana.,COP,47990,2023-09-04T22:04:02Z,https://www.tiendasjumbo.co/arroz-diana-x-25-und-x-500-g-c-u/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/186291/7702511000342.jpg?v=1751416139,Cencosud - Colombia,[ x 25 und x 500 g c-u],2022-10-05,ARROZ
7,13381,Arroz Diana x 1 Kg,DIANA,Proporciónale todo el amor a tu familia con el delicioso Arroz Diana,COP,3990,2023-08-28T22:12:42Z,https://www.tiendasjumbo.co/arroz-diana-x-1-kg/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/186299/7702511000021.jpg?v=1751416045,Cencosud - Colombia,[ x 1 Kg],2022-10-05,ARROZ
8,111864,Arroz Cuisine&amp;Co blanco x5000 g,CUISINE &amp; CO NBE MP,Arroz cuisine&amp;co blanco x5000g,COP,18790,2022-10-17T05:00:00Z,https://www.tiendasjumbo.co/arroz-cuisine-co-blanco-x5000g/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/213564/7703616275024.jpg?v=1751416110,Cencosud - Colombia,[ x5000 g],2022-10-05,ARROZ
9,13389,Arroz Roa x 10 kg,ROA,arroz descascarado al cual se le ha eliminado totalmente por elaboracion el germen y el salvado con adicion de vitamina A y acido folico,COP,44590,2023-08-29T22:01:46Z,https://www.tiendasjumbo.co/arroz-roa-x-10kg/p,https://jumbocolombiaio.vtexassets.com/arquivos/ids/186308/7702552000172.jpg?v=1751416108,Cencosud - Colombia,[ x 10 kg],2022-10-05,ARROZ


In [165]:
producto = 'Aceite Riquisimo garrafa 5000 ml'
cantidadx(producto)

'5000 ml'

In [ ]:
producto = 'Arroz vixtarroz plusx Supdremo x 10 kg.'


producto=producto[::-1] #Invertimos la cadena para encontrar x
#m =producto.find("x")
#producto = producto[:m] #Esta invertimo el valor


producto = re.findall(r'(\d + x)',producto)

#producto=producto[::-1] #Lo volvemos a su estado normal
producto

[]

In [ ]:
Productos_T

NameError: name 'Productos_T' is not defined

In [76]:


# Ver todo el contenido de un dataframe
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.options.display.max_colwidth = None

In [103]:
m = Productos_T['Cantidad']
m

0                       10 kg
1                      10000g
2                       10 Kg
3                        5 Kg
4                        5 kg
5                        3 Kg
6           25 und  500 g c-u
7                        1 Kg
8                       5000g
9                        10kg
10                        5kg
11                      10 kg
12            25 und 500g c-u
13                      1000g
14                     4000 g
15                        5kg
16                      3000g
17                     3000 g
18                     2.5 kg
19                      10 kg
20                     2.5 Kg
21                      4000g
22                      1000g
23                       5 kg
24                       3 kg
25                     1000 g
26                     1000 g
27                       5 kg
28                       10kg
29                       500g
30                      3000g
31                      3000g
32                      1000g
33        

In [17]:
#Productos_T['Nombre']